<a href="https://colab.research.google.com/github/dongchanlim/CS450/blob/master/Week_11_Ensemble_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from google.colab import files
file = files.upload()

Saving LasVegasTripAdvisorReviews-Dataset.csv to LasVegasTripAdvisorReviews-Dataset (2).csv


In [69]:
trip = pd.read_csv("LasVegasTripAdvisorReviews-Dataset.csv", sep = ';', skipinitialspace= True, na_values= ["?"])

In [70]:
trip.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


In [71]:
trip.dtypes

User country         object
Nr. reviews           int64
Nr. hotel reviews     int64
Helpful votes         int64
Score                 int64
Period of stay       object
Traveler type        object
Pool                 object
Gym                  object
Tennis court         object
Spa                  object
Casino               object
Free internet        object
Hotel name           object
Hotel stars          object
Nr. rooms             int64
User continent       object
Member years          int64
Review month         object
Review weekday       object
dtype: object

In [72]:
# See target (Class) variables and distribution
trip['Hotel name'].value_counts()

Excalibur Hotel & Casino                               24
The Venetian Las Vegas Hotel                           24
Monte Carlo Resort&Casino                              24
Treasure Island- TI Hotel & Casino                     24
Encore at wynn Las Vegas                               24
Tuscany Las Vegas Suites & Casino                      24
Wynn Las Vegas                                         24
Hilton Grand Vacations at the Flamingo                 24
The Palazzo Resort Hotel Casino                        24
Bellagio Las Vegas                                     24
Circus Circus Hotel & Casino Las Vegas                 24
Hilton Grand Vacations on the Boulevard                24
Caesars Palace                                         24
Trump International Hotel Las Vegas                    24
Tropicana Las Vegas - A Double Tree by Hilton Hotel    24
Marriott's Grand Chateau                               24
Paris Las Vegas                                        24
Wyndham Grand 

In [74]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

num_col = trip.select_dtypes(["number"]).columns

trip[num_col] = std.fit_transform(trip[num_col])

In [75]:
trip.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,-0.495595,-0.502370,-0.386857,0.871491,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,1.227704,North America,0.102120,January,Thursday
1,USA,0.945905,0.207912,0.892215,-1.115982,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,1.227704,North America,0.027690,January,Friday
2,USA,-0.161914,-0.293464,-0.139295,0.871491,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,1.227704,North America,0.015285,February,Saturday
3,UK,-0.455553,-0.377026,-0.366227,-0.122245,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,1.227704,Europe,0.064905,February,Friday
4,Canada,-0.575678,-0.460589,-0.613789,-0.122245,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,1.227704,North America,0.077310,March,Tuesday


In [77]:
trip.shape

(504, 20)

In [76]:
trip.isnull().sum(axis = 0) / trip.shape[0] * 100

User country         0.0
Nr. reviews          0.0
Nr. hotel reviews    0.0
Helpful votes        0.0
Score                0.0
Period of stay       0.0
Traveler type        0.0
Pool                 0.0
Gym                  0.0
Tennis court         0.0
Spa                  0.0
Casino               0.0
Free internet        0.0
Hotel name           0.0
Hotel stars          0.0
Nr. rooms            0.0
User continent       0.0
Member years         0.0
Review month         0.0
Review weekday       0.0
dtype: float64

In [78]:
from sklearn.model_selection import train_test_split

X = trip.drop(columns = 'Hotel name')
y = trip['Hotel name'].astype("category").cat.codes

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

1. Try at least 3 different "regular" learning algorithms and note the results.

In [86]:
# KNN classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors = np.int((np.sqrt(trip.size))), weights = "distance", n_jobs = -1)

knn.fit(X_train, y_train)

predicted_y = knn.predict(X_test)

accuracy_score(predicted_y, y_test)

0.4934210526315789

In [87]:
# Naive Bayesian
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train, y_train)

predicted_y = nb.predict(X_test)

accuracy_score(predicted_y, y_test)

0.8947368421052632

In [88]:
# SVM
from sklearn.svm import SVC

svm = SVC(gamma='auto', kernel = 'rbf')

svm.fit(X_train, y_train)

predicted_y = svm.predict(X_test)

accuracy_score(predicted_y, y_test)

0.3355263157894737

2. Use a random forest and note the results. (Play around with a few different options)


In [89]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 150, criterion = 'entropy', max_depth = 4, n_jobs = -1, random_state = 0, class_weight = "balanced_subsample")

rfc.fit(X_train, y_train)

# predicted_y = rfc.predict(X_test)
# accuracy_score(predicted_y, y_test)

rfc.score(X_test, y_test)

0.875

In [90]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 150, 200], 'criterion': ["gini",'entropy'], 'max_depth': [2,3,4,5], 'class_weight': ['balanced', 'balanced_subsample'], 'n_jobs' : [-1]}

search = GridSearchCV(rfc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 4,
 'n_estimators': 150,
 'n_jobs': -1}

In [91]:
search.best_score_

0.8637424547283702

3. Use a gradient boosting machine (GBM) and note the results. (Play around with a few different options)


In [96]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.5, max_depth= 4, random_state= 0)

gbc.fit(X_train, y_train)

gbc.score(X_test, y_test)

1.0

In [97]:
param_grid = {'criterion': ['friedman_mse', 'mse', 'mae'], 'loss' :['deviance', 'exponential'], 'max_features':['auto', 'sqrt', 'log2'], 'max_depth': [2,3,4,5]}

search = GridSearchCV(gbc, param_grid, cv=5)

search.fit(X_train, y_train)

search.best_params_

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 21 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 21 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: ExponentialLoss requires 2 classes; got 21 class(es)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Es

{'criterion': 'friedman_mse',
 'loss': 'deviance',
 'max_depth': 2,
 'max_features': 'auto'}

In [98]:
search.best_score_

1.0